<a href="https://colab.research.google.com/github/vijayrowtula/colab_notebooks/blob/main/MNIST_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Practise notebook created using code from https://github.com/AvivSham/Pytorch-MNIST-colab. Thanks to original author!

In [1]:
# install relavant libraries
# !pip install torch
# !pip install torchvision

In [2]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [3]:
#@title Hyperparameters

input_size = 784 # img_size = (28,28) ---> 28*28=784 in total
hidden_size = 500 # number of nodes at hidden layer
num_classes = 10 # number of output classes discrete range [0,9]
num_epochs = 20 # number of times which the entire dataset is passed throughout the model
batch_size = 100 # the size of input data took for one iteration
lr = 1e-3 # size of step 

In [4]:
#@title Downloading MNIST data

train_data = datasets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = datasets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
#@title Load the data

train_gen = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size,shuffle=True)
test_gen = torch.utils.data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

In [6]:
#@title Model

class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [7]:
# build model
model = Net(input_size, hidden_size, num_classes)
if torch.cuda.is_available():
  print('GPU available')
  model.cuda()

GPU available


In [9]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [10]:
model.parameters()

<generator object Module.parameters at 0x7fe4b4e529d0>

In [12]:
#@title Training the model
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_gen):
    images = Variable(images.view(-1,28*28)).cuda()
    labels = Variable(labels).cuda()

    optimizer.zero_grad()
    outputs = model(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_data)//batch_size, loss.data))

Epoch [1/20], Step [100/600], Loss: 0.3892
Epoch [1/20], Step [200/600], Loss: 0.2684
Epoch [1/20], Step [300/600], Loss: 0.1754
Epoch [1/20], Step [400/600], Loss: 0.1103
Epoch [1/20], Step [500/600], Loss: 0.2322
Epoch [1/20], Step [600/600], Loss: 0.1545
Epoch [2/20], Step [100/600], Loss: 0.0948
Epoch [2/20], Step [200/600], Loss: 0.0598
Epoch [2/20], Step [300/600], Loss: 0.0854
Epoch [2/20], Step [400/600], Loss: 0.1165
Epoch [2/20], Step [500/600], Loss: 0.0383
Epoch [2/20], Step [600/600], Loss: 0.0578
Epoch [3/20], Step [100/600], Loss: 0.0733
Epoch [3/20], Step [200/600], Loss: 0.1358
Epoch [3/20], Step [300/600], Loss: 0.0354
Epoch [3/20], Step [400/600], Loss: 0.0880
Epoch [3/20], Step [500/600], Loss: 0.0905
Epoch [3/20], Step [600/600], Loss: 0.0300
Epoch [4/20], Step [100/600], Loss: 0.1285
Epoch [4/20], Step [200/600], Loss: 0.0460
Epoch [4/20], Step [300/600], Loss: 0.0363
Epoch [4/20], Step [400/600], Loss: 0.1256
Epoch [4/20], Step [500/600], Loss: 0.0756
Epoch [4/20

In [13]:
#@title Evaluating the accuracy of the model

correct = 0
total = 0
for images,labels in test_gen:
  images = Variable(images.view(-1,28*28)).cuda()
  labels = labels.cuda()
  
  output = model(images)
  _, predicted = torch.max(output,1)
  correct += (predicted == labels).sum()
  total += labels.size(0)

print('Accuracy of the model: %.3f %%' %((100*correct)/(total+1)))

Accuracy of the model: 97.930 %
